![](https://numer.ai/img/Numerai-Logo-Side-Black.8393ed16.png)

（履歴）
- version 3: 公開
- version 4: タイポなど修正
- version 5: target nomiへ変更。加筆。

「Numeraiはじめてみたいけど、英語ドキュメントが多くてちょっと...」という方も多いと思いますので、日本語で「データのロード→モデリング→評価→提出」の一連の流れをまとめてみました。

ただ、最近Numerai関連の日本語ドキュメントが充実してきまして、公式でまとまってるので一度ご覧になると良いかも知れないです。

[Numerai日本語公式ドキュメント](https://jp.docs.numer.ai/numerai-tournament/new-users)

それではやっていきましょう。

まず、NumeraiにはAPIが用意されていますので、利用するためのモジュールをインストールします。

他の必要なライブラリをインストールします。

In [4]:
import gc
import operator
import os
import pathlib
import sys
from typing import Callable, Dict, Generic, List, NoReturn, Optional, Text, Tuple, Union

import joblib

# model
import lightgbm as lgb

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import numerapi
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
from scipy.stats import spearmanr
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    log_loss,
    mean_absolute_error,
    mean_squared_error,
    roc_auc_score,
)
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    QuantileTransformer,
    StandardScaler,
)

sns.set_context("talk")
style.use("seaborn-colorblind")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Tournamentデータのダウンロード
AWSに公開されていますので、誰でも以下のようにダウンロード可能です。データの特徴量は離散値になっているので、メモリ節約のため整数値にキャストしています。Round期間を表す'Era'も整数にします。

In [5]:
def get_int(x):
    try:
        return int(x[3:])
    except:

        return 1000


def read_data(data="train"):
    # get data
    if data == "train":
        df = pd.read_csv(
            "https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz"
        )
    elif data == "test":
        df = pd.read_csv(
            "https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz"
        )

    # features
    feature_cols = df.columns[df.columns.str.startswith("feature")]

    # map to int, to reduce the memory demand
    mapping = {0.0: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.0: 4}
    for c in feature_cols:
        df[c] = df[c].map(mapping).astype(np.uint8)

    # also cast era to int
    df["era"] = df["era"].apply(get_int)
    return df

In [ ]:
%%time

# load train　(半年間固定)
train = read_data("train")
print(train.shape)
train.head()

In [ ]:
%%time

# load test (毎週Roundごとに更新)
test = read_data("test")

Tournamentデータには、ラベルが与えられているValidationデータと、ラベルがあたえられていないTestデータが入っているので、分離します。また、Validationデータは主に2期間に分かれている（前の期間は予測が簡単で、後の方が予測が難しい）ため、ここを区別したラベルを付与します。

In [ ]:
valid = test[test["data_type"] == "validation"].reset_index(drop=True)

# validation split
valid.loc[valid["era"] > 180, "valid2"] = True  # むずいやつ
valid.loc[valid["era"] <= 180, "valid2"] = False  # 簡単なやつ

In [ ]:
# remove data_type to save memory
train.drop(columns=["data_type"], inplace=True)
valid.drop(columns=["data_type"], inplace=True)
test.drop(columns=["data_type"], inplace=True)

print(
    "The number of records: train {:,}, valid {:,}, test {:,}".format(
        train.shape[0], valid.shape[0], test.shape[0]
    )
)

# EDA (Exploratory Data Analysis)の実行
簡単にですが、どんなデータなのか見てみましょう。

## 特徴量

In [ ]:
# features (特徴量)
features = [f for f in train.columns.values.tolist() if "feature" in f]
print("There are {} features.".format(len(features)))
print(features)

310個の匿名特徴量がありますね。

- intelligence (1 ~ 12)
- charisma (1 ~ 86)
- strength (1 ~ 38)
- dexterity (1 ~ 14)
- constitution (1 ~ 114)
- wisdom (1 ~ 46)

と6種類に大別できそうですが、それぞれどういった特徴量なのかはわかりません。

## Target
targetはどうでしょうか。

In [ ]:
# target
target = train.columns[train.columns.str.startswith("target")].values.tolist()[0]
print(f"Taget name = {target}")

In [ ]:
train[target].hist()

正規分布っぽい5つの離散値であることがわかります。

# Modelingの実行
データの整理ができたので、[公式Example](https://github.com/numerai/example-scripts/blob/master/example_model.py)にならいモデリングをしていきます。

[公式ではXGBoostを使用](https://jp.docs.numer.ai/numerai-tournament/tournament-overview)していますが、時短のためここではLightGBMを使用します。

XGBoostもLightGBMもGBDT (Gradient Boosting Decision Tree)と呼ばれる類のモデルで、テーブル形式のデータに大して非常に強力です。パラメータが複数あってどうしたらいいかわからない方は、こちらのブログに非常に簡潔にわかりやすくまとまっていますのでオススメです。

[勾配ブースティングで大事なパラメータの気持ち](https://nykergoto.hatenablog.jp/entry/2019/03/29/%E5%8B%BE%E9%85%8D%E3%83%96%E3%83%BC%E3%82%B9%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0%E3%81%A7%E5%A4%A7%E4%BA%8B%E3%81%AA%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%81%AE%E6%B0%97%E6%8C%81%E3%81%A1)

In [ ]:
# # create a model and fit (公式example)
# model = xgb.XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1)
# model.fit(train[features], train[target])

In [ ]:
%%time

# create a model and fit（LGBのハイパラは↑の公式XGBに寄せてみました）
params = {
    "n_estimators": 2000,
    "objective": "regression",
    "boosting_type": "gbdt",
    "max_depth": 5,
    "learning_rate": 0.01,
    "feature_fraction": 0.1,
    "seed": 42,
}
model = lgb.LGBMRegressor(**params)
model.fit(train[features], train[target])

In [ ]:
# save model（あとでロードして予測できるように保存します）
joblib.dump(model, "my_lightgbm.joblib")
print("model saved!")

# Feature importance
せっかくGBDTを使っているので、どの特徴量が大事か見てみましょう。

In [ ]:
pd.DataFrame(model.feature_importances_, index=features, columns=["importance"]).sort_values(
    by="importance", ascending=False
).style.background_gradient(cmap="viridis")

特徴量は全て匿名なので、それぞれ具体的に何を意味しているかはわかりませんが、少なくとも特別強い、あるいは特別弱い特徴量というのはなさそうです。

# Validation Score
Validationデータが与えられているので、訓練したモデルがどの程度のものか、スコアを計算してみましょう。金融モデルなので、ただ精度 (targetとのrank correlation)だけでなく、運用期間で安定したパフォーマンスを出せているかチェックします。多くの関数はNumeraiの[公式Github](https://github.com/numerai/example-scripts/blob/master/example_model.py)にあるので、拾って改変し使っていきます。

MMC（meta model correlation）は、運営のメタモデルが手に入らないため計算できませんが、精度 (rank correlation)と、精度と安定性のバランスを評価する**correlation sharpe**などは、自分で計算できますので計算しましょう。以下の指標を、Validation期間を全て、前半（簡単なやつ）、後半（難しいやつ）に分けて計算しています。

- rank correlation (NumeraiでCORRと呼ばれているもの。高いほど良い)
- sharpe ratio（期間ベースでのCORR平均を標準偏差で割ったもの。高いほど良い）
- max drawdown (ある1ラウンドでの最大の損失CORR。0に近いほど良い)
- feature exposure (モデル予測値が一部の特徴量に依存している度合。低いほど良い)

実は提出すると、Numerai側で全部計算して自分のページで確認することができるのですが、全validation期間を使用したスコアのみが返ってくるため、「予測が難しい時期でもいいパフォーマンスが出ているか？」という肝心の疑問には答えてくれません。なので、自分で計算しましょう...^^

In [ ]:
# naming conventions
PREDICTION_NAME = "prediction"
TARGET_NAME = target
# EXAMPLE_PRED = 'example_prediction'

# ---------------------------
# Functions
# ---------------------------
def valid4score(
    valid: pd.DataFrame, pred: np.ndarray, load_example: bool = True, save: bool = False
) -> pd.DataFrame:
    """
    Generate new valid pandas dataframe for computing scores

    :INPUT:
    - valid : pd.DataFrame extracted from tournament data (data_type='validation')

    """
    valid_df = valid.copy()
    valid_df["prediction"] = pd.Series(pred).rank(pct=True, method="first")
    valid_df.rename(columns={target: "target"}, inplace=True)

    if load_example:
        valid_df[EXAMPLE_PRED] = pd.read_csv(EXP_DIR + "valid_df.csv")["prediction"].values

    if save == True:
        valid_df.to_csv(OUTPUT_DIR + "valid_df.csv", index=False)
        print("Validation dataframe saved!")

    return valid_df


def compute_corr(valid_df: pd.DataFrame):
    """
    Compute rank correlation

    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist

    """

    return np.corrcoef(valid_df["target"], valid_df["prediction"])[0, 1]


def compute_max_drawdown(validation_correlations: pd.Series):
    """
    Compute max drawdown

    :INPUT:
    - validation_correaltions : pd.Series
    """

    rolling_max = (validation_correlations + 1).cumprod().rolling(window=100, min_periods=1).max()
    daily_value = (validation_correlations + 1).cumprod()
    max_drawdown = -(rolling_max - daily_value).max()

    return max_drawdown


def compute_val_corr(valid_df: pd.DataFrame):
    """
    Compute rank correlation for valid periods

    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """

    # all validation
    correlation = compute_corr(valid_df)
    print("rank corr = {:.4f}".format(correlation))
    return correlation


def compute_val_sharpe(valid_df: pd.DataFrame):
    """
    Compute sharpe ratio for valid periods

    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    d = valid_df.groupby("era")[["target", "prediction"]].corr().iloc[0::2, -1].reset_index()
    me = d["prediction"].mean()
    sd = d["prediction"].std()
    max_drawdown = compute_max_drawdown(d["prediction"])
    print(
        "sharpe ratio = {:.4f}, corr mean = {:.4f}, corr std = {:.4f}, max drawdown = {:.4f}".format(
            me / sd, me, sd, max_drawdown
        )
    )

    return me / sd, me, sd, max_drawdown


def feature_exposures(valid_df: pd.DataFrame):
    """
    Compute feature exposure

    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    feature_names = [f for f in valid_df.columns if f.startswith("feature")]
    exposures = []
    for f in feature_names:
        fe = spearmanr(valid_df["prediction"], valid_df[f])[0]
        exposures.append(fe)
    return np.array(exposures)


def max_feature_exposure(fe: np.ndarray):
    return np.max(np.abs(fe))


def feature_exposure(fe: np.ndarray):
    return np.sqrt(np.mean(np.square(fe)))


def compute_val_feature_exposure(valid_df: pd.DataFrame):
    """
    Compute feature exposure for valid periods

    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    fe = feature_exposures(valid_df)
    fe1, fe2 = feature_exposure(fe), max_feature_exposure(fe)
    print("feature exposure = {:.4f}, max feature exposure = {:.4f}".format(fe1, fe2))

    return fe1, fe2


# to neutralize a column in a df by many other columns
def neutralize(df, columns, by, proportion=1.0):
    scores = df.loc[:, columns]
    exposures = df[by].values

    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack(
        (
            exposures,
            np.asarray(np.mean(scores)) * np.ones(len(exposures)).reshape(-1, 1),
        )
    )

    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()


# to neutralize any series by any other series
def neutralize_series(series, by, proportion=1.0):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)

    # this line makes series neutral to a constant column so that it's centered and for sure gets corr 0 with exposures
    exposures = np.hstack((exposures, np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))

    correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores, rcond=None)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized


def unif(df):
    x = (df.rank(method="first") - 0.5) / len(df)
    return pd.Series(x, index=df.index)


def get_feature_neutral_mean(df):
    feature_cols = [c for c in df.columns if c.startswith("feature")]
    df.loc[:, "neutral_sub"] = neutralize(df, [PREDICTION_NAME], feature_cols)[PREDICTION_NAME]
    scores = (
        df.groupby("era")
        .apply(
            lambda x: np.corrcoef(x["neutral_sub"].rank(pct=True, method="first"), x[TARGET_NAME])
        )
        .mean()
    )
    return np.mean(scores)


def compute_val_mmc(valid_df: pd.DataFrame):
    # MMC over validation
    mmc_scores = []
    corr_scores = []
    for _, x in valid_df.groupby("era"):
        series = neutralize_series(
            pd.Series(unif(x[PREDICTION_NAME])), pd.Series(unif(x[EXAMPLE_PRED]))
        )
        mmc_scores.append(np.cov(series, x[TARGET_NAME])[0, 1] / (0.29 ** 2))
        corr_scores.append(
            np.corrcoef(unif(x[PREDICTION_NAME]).rank(pct=True, method="first"), x[TARGET_NAME])
        )

    val_mmc_mean = np.mean(mmc_scores)
    val_mmc_std = np.std(mmc_scores)
    val_mmc_sharpe = val_mmc_mean / val_mmc_std
    corr_plus_mmcs = [c + m for c, m in zip(corr_scores, mmc_scores)]
    corr_plus_mmc_sharpe = np.mean(corr_plus_mmcs) / np.std(corr_plus_mmcs)
    corr_plus_mmc_mean = np.mean(corr_plus_mmcs)

    print(
        "MMC Mean = {:.6f}, MMC Std = {:.6f}, CORR+MMC Sharpe = {:.4f}".format(
            val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe
        )
    )

    # Check correlation with example predictions
    corr_with_example_preds = np.corrcoef(
        valid_df[EXAMPLE_PRED].rank(pct=True, method="first"),
        valid_df[PREDICTION_NAME].rank(pct=True, method="first"),
    )[0, 1]
    print("Corr with example preds: {:.4f}".format(corr_with_example_preds))

    return val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe, corr_with_example_preds


def score_summary(valid_df: pd.DataFrame):
    score_df = {}

    try:
        score_df["correlation"] = compute_val_corr(valid_df)
    except:
        print("ERR: computing correlation")
    try:
        (
            score_df["corr_sharpe"],
            score_df["corr_mean"],
            score_df["corr_std"],
            score_df["max_drawdown"],
        ) = compute_val_sharpe(valid_df)
    except:
        print("ERR: computing sharpe")
    try:
        (
            score_df["feature_exposure"],
            score_df["max_feature_exposure"],
        ) = compute_val_feature_exposure(valid_df)
    except:
        print("ERR: computing feature exposure")
    try:
        (
            score_df["mmc_mean"],
            score_df["mmc_std"],
            score_df["corr_mmc_sharpe"],
            score_df["corr_with_example_xgb"],
        ) = compute_val_mmc(valid_df)
    except:
        print("ERR: computing MMC")

    return pd.DataFrame.from_dict(score_df, orient="index")

In [ ]:
# prediction for valid periods
pred = model.predict(valid[features])

In [ ]:
# scores
valid_df = valid4score(valid, pred, load_example=False, save=False)

score_df = pd.DataFrame()
print("------------------")
print("ALL:")
print("------------------")
all_ = score_summary(valid_df).rename(columns={0: "all"})

print("------------------")
print("VALID 1:")
print("------------------")
val1_ = score_summary(valid_df.query("era < 150")).rename(columns={0: "val1"})

print("------------------")
print("VALID 2:")
print("------------------")
val2_ = score_summary(valid_df.query("era > 150")).rename(columns={0: "val2"})

In [ ]:
# scores
score_df = pd.concat([all_, val1_, val2_], axis=1)
score_df.style.background_gradient(cmap="viridis", axis=0)

明らかにVALID2（予測が難しいValidation期間）の方が、CORRが小さかったりと難しいですね。

ただ、valid 2の予測が難しい期間でもCORR = 0.015あたりのスコアなので、**RoundでCORRのみにBetすれば（週次）1.5%のリターンが期待できそう**だということになります。CORR+MMCにBetすれば、（MMCがプラスなら）更なるリターンが見込めるモデルになっています。

もちろん相場の動きは気まぐれです。これで「絶対に儲かる...!」というものではないので、各自データサイエンティストとして腕の見せ所です。

# Submission
以下提出の形式です。実際に提出するには、[Numerai tournament](https://numer.ai/tournament)でユーザ登録を行い、APIキーとモデルIDを取得してください。Rank correlationで評価されるため、こちらでrank化して提出します。

In [ ]:
public_id = "NYANNYAN"  # replace with yours
secret_key = "WANWAN"  # replace with yours
model_id = "KOKEKOKKOOOO"  # replace with yours
PREDICTION_NAME = "prediction_kazutsugi"  # 現在はこれ（いずれprediction_nomiになるらしい）
OUTPUT_DIR = ""  # prediction dataframeを保存するpath


def submit(tournament: pd.DataFrame, pred: np.ndarray, model_id="abcde"):
    predictions_df = tournament["id"].to_frame()
    predictions_df[PREDICTION_NAME] = pred

    # to rank
    predictions_df[PREDICTION_NAME] = predictions_df[PREDICTION_NAME].rank(
        pct=True, method="first"
    )

    # save
    predictions_df.to_csv(pathlib.Path(OUTPUT_DIR + f"predictions_{model_id}.csv"), index=False)

    # Upload your predictions using API
    napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)
    submission_id = napi.upload_predictions(
        pathlib.Path(OUTPUT_DIR + f"predictions_{model_id}.csv"), model_id=model_id
    )
    print("submitted to {model_id}", model_id=model_id)

    return predictions_df

In [ ]:
# prediction
pred = model.predict(test[features])
plt.hist(pred);

In [ ]:
# submit!（本当に提出する人はコメントアウトしてください）
# predictions_df = submit(tournament, pred, model_id=model_id)

# 終わりに
Numeraiは英語の情報が多く、とっつきにくかった人も多いとは思いますが、このNotebookがみなさんのNumerai lifeの参考になれば幸いです。

# 参考

- [KagglerへのNumeraiのススメ](https://zenn.dev/katsu1110/articles/bb2b5cba9b04c9e30bfe)
- [Numerai公式Github](https://github.com/numerai)